# Prepare data and create the model

Now we have data that mimics what we might have after a company has created a Customer 360. We have demographic data and a total of all orders the customer has placed, where that total is a function of the demographic data we have gathered. Next, we will create a view which will represent the data used to train the value, then we will create the model.

## Create training data view

We run the following to create a view which limits the columns selected (we do not use the email address or the join date), and we limit only to the longer-term customers who will be used to train the model. Finally, we bucket customers into bronze, silver, and gold groups using the total order value column.

In [ ]:
from snowflake.snowpark.context import get_active_session

In [ ]:
session = get_active_session()

In [ ]:
-- create a view to train the model
create or replace view customer_training
as select age_band, household_income, marital_status, household_size, case when total_order_value<10 then 'BRONZE'
    when total_order_value<=25 and total_order_value>10 then 'SILVER'
    else 'GOLD' END as segment
from customers
where join_date<'2024-02-11'::date;

In [ ]:
-- verify the training view
select * from customer_training LIMIT 10;

## Build the model
We can create the classification model by running the following statement.

In [ ]:
-- create the classification model
CREATE OR REPLACE SNOWFLAKE.ML.CLASSIFICATION customer_classification_model(
    INPUT_DATA => SYSTEM$REFERENCE('view', 'customer_training'),
    TARGET_COLNAME => 'segment'
);


Notice that we point the model at the view we created, and we specify the column containing the bronze, silver, or gold segmentation. We can verify that the model is created by running the following.

In [ ]:
SHOW SNOWFLAKE.ML.CLASSIFICATION;

## Save the model predictions to a table

Next, we will run the predictions across our entire customers table, saving the results to a table with the email address, allowing us to join later, when we use the predictions to build audiences.


In [ ]:
-- run prediction and save results
CREATE OR REPLACE TABLE customer_predictions AS
SELECT email, customer_classification_model!PREDICT(INPUT_DATA => object_construct(*)) as predictions
from customers;

Next we can run the following query to verify that our predictions have been made correctly.


In [ ]:
-- verify the created predictions
SELECT * FROM customer_predictions LIMIT 10;

## Creating audiences

Once we have run these predictions, we can use the predictions to build audiences. In some flows, the audiences could be saved, or a reverse ETL tool might query for them directly. The email addresses could be used to activate the audiences across both paid (social, CTV or programmatic platforms) or owned (email, website) marketing channels.

## High-value new customers

The following query can now be run to create a list of new customers who are likely to become "GOLD" customers in the future.


In [ ]:
-- new customers likely to be gold
select c.email
from customers c
 inner join customer_predictions p on c.email=p.email
where c.join_date>='2024-02-11'::date and predictions:class='GOLD'
limit 10;

If we wanted to customize the confidence at which we want to consider them likely gold customers, we could use the probabilities saved in the predictions.

This list of users might be a list we consider worthy of additional marketing, and we may be willing to pay more to show ads to this audience. We also might email this group a coupon code, for instance, to drive to drive additional purchases in the hope that they become regular, loyal customers.

## Underutilized existing customers

The usage of the model is not limited only to new customers. We can also use the model to find existing customers who, although they are not gold, are similar to customers who are. This could indicate an opportunity to reach out these customers across channels to drive additional purchases.


In [ ]:
-- old customers who are not gold but should be
select c.email
from customers c
 inner join customer_predictions p on c.email=p.email
where c.join_date<'2024-02-11'::date and predictions:class='GOLD'
 and c.total_order_value<=25;